# Water Quality Prediction — Submission Notebook

In [ ]:
!pip install uv
!uv pip install -r requirements.txt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.stats import mstats
from sklearn.cluster import KMeans
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.inspection import permutation_importance

## 1. Load & Merge Data (join by key, not by index)

In [ ]:
wq = pd.read_csv("water_quality_training_dataset.csv")
landsat = pd.read_csv("landsat_features_training.csv")
terra = pd.read_csv("terraclimate_features_training.csv")

# Merge by key — never by index
JOIN_KEYS = ['Latitude', 'Longitude', 'Sample Date']
df = wq.merge(landsat, on=JOIN_KEYS, how='left') \
       .merge(terra, on=JOIN_KEYS, how='left')

print(f"Shape after merge: {df.shape}")
display(df.head())

## 2. Landsat Calibration (DN → Reflectance) & Recalculate Indices

In [ ]:
BANDS = ['nir', 'green', 'swir16', 'swir22']

def calibrate_landsat(data):
    """Apply Landsat C2L2 scale/offset to convert DN → surface reflectance."""
    scale, offset = 0.0000275, -0.2
    for col in BANDS:
        data[col] = data[col] * scale + offset
    # Recalculate indices from calibrated reflectance
    data['NDMI']  = (data['nir'] - data['swir16']) / (data['nir'] + data['swir16'])
    data['MNDWI'] = (data['green'] - data['swir22']) / (data['green'] + data['swir22'])
    return data

df = calibrate_landsat(df)
print("Calibrated reflectance ranges:")
for b in BANDS:
    print(f"  {b}: [{df[b].min():.4f}, {df[b].max():.4f}]")

## 3. Cloud Filter, Cluster-based Imputation & DRP Censoring

In [ ]:
# Cloud filter: after calibration, valid reflectance is roughly [0, 1.0]
for col in BANDS:
    df[col] = df[col].where(df[col].between(-0.05, 1.2), np.nan)

# Cluster-based imputation (spatial, not global median)
N_CLUSTERS = 8
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
df['geo_cluster'] = kmeans.fit_predict(df[['Latitude', 'Longitude']])

impute_cols = BANDS + ['NDMI', 'MNDWI', 'pet']
for col in impute_cols:
    cluster_med = df.groupby('geo_cluster')[col].transform('median')
    df[col] = df[col].fillna(cluster_med)
# Fallback: global median for any remaining NaN
train_medians = df[impute_cols].median()
df[impute_cols] = df[impute_cols].fillna(train_medians)

# DRP censoring: 10.0 = below detection limit → replace with DL/2
df['Dissolved Reactive Phosphorus'] = df['Dissolved Reactive Phosphorus'].replace(10.0, 5.0)

print(f"Remaining NaNs:\n{df.isna().sum()[df.isna().sum() > 0]}")

## 4. Feature Engineering

In [ ]:
dates = pd.to_datetime(df['Sample Date'], dayfirst=True)
df['month_sin'] = np.sin(2 * np.pi * dates.dt.month / 12)
df['month_cos'] = np.cos(2 * np.pi * dates.dt.month / 12)
df['NIR_SWIR22_ratio'] = df['nir'] / (df['swir22'] + 1e-6)
df['turbidity']        = df['swir16'] / (df['green'] + 1e-6)

# KNN is distance-based: year creates artificial distance between identical
# seasonal patterns and PET lags are mostly copies of pet (first obs fallback),
# adding redundant dimensions that hurt KNN. Removed both.
FEATURE_COLS = [
    'nir', 'green', 'swir16', 'swir22',
    'NDMI', 'MNDWI',
    'pet',
    'month_sin', 'month_cos',
    'NIR_SWIR22_ratio', 'turbidity',
]
print(f"Features ({len(FEATURE_COLS)}): {FEATURE_COLS}")

## 5. Winsorize Targets & Prepare Training Data

In [ ]:
TARGETS = ['Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']

# Winsorize extreme outliers before log transform (critical for KNN — outliers distort distances)
for col in TARGETS:
    df[col] = mstats.winsorize(df[col], limits=[0.01, 0.01])

# Groups for spatial CV
groups = df[['Latitude', 'Longitude']].apply(lambda r: f"{r['Latitude']}_{r['Longitude']}", axis=1)

X = df[FEATURE_COLS]
y_TA_log  = np.log1p(df['Total Alkalinity'])
y_EC_log  = np.log1p(df['Electrical Conductance'])
y_DRP_log = np.log1p(df['Dissolved Reactive Phosphorus'])

print(f"Samples: {len(X)} | Unique locations: {groups.nunique()}")

## 6. Model Training (KNN + StandardScaler + GroupKFold)

In [ ]:
def run_pipeline(X, y_log, groups, param_name, n_splits=5):
    print(f"\n{'='*60}\nTraining: {param_name}\n{'='*60}")

    group_kfold = GroupKFold(n_splits=n_splits)

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', KNeighborsRegressor(weights='distance', metric='euclidean'))
    ])

    param_grid = {
        'model__n_neighbors': [5, 7, 10, 15, 20, 30],
        'model__p': [1, 2],  # 1=manhattan, 2=euclidean
    }

    rmse_scorer = make_scorer(
        lambda yt, yp: np.sqrt(mean_squared_error(np.expm1(yt), np.expm1(yp))),
        greater_is_better=False)
    r2_scorer = make_scorer(
        lambda yt, yp: r2_score(np.expm1(yt), np.expm1(yp)),
        greater_is_better=True)

    search = GridSearchCV(
        pipeline, param_grid, cv=group_kfold,
        scoring={'r2': r2_scorer, 'rmse': rmse_scorer}, refit='r2',
        n_jobs=-1, verbose=1, return_train_score=True)

    search.fit(X, y_log, groups=groups)

    best = search.best_estimator_
    idx = search.best_index_
    cv = search.cv_results_

    print(f"\nBest params: {search.best_params_}")
    print(f"CV Train R²: {cv['mean_train_r2'][idx]:.4f} | Val R²: {cv['mean_test_r2'][idx]:.4f}")
    print(f"CV Train RMSE: {-cv['mean_train_rmse'][idx]:.4f} | Val RMSE: {-cv['mean_test_rmse'][idx]:.4f}")

    return best

model_TA  = run_pipeline(X, y_TA_log,  groups, "Total Alkalinity")
model_EC  = run_pipeline(X, y_EC_log,  groups, "Electrical Conductance")
model_DRP = run_pipeline(X, y_DRP_log, groups, "Dissolved Reactive Phosphorus")

## 7. Permutation Feature Importance

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for ax, model, y_log, name in zip(axes,
        [model_TA, model_EC, model_DRP],
        [y_TA_log, y_EC_log, y_DRP_log],
        ['Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']):
    pi = permutation_importance(model, X, y_log, n_repeats=15, random_state=42, n_jobs=-1)
    sorted_idx = pi.importances_mean.argsort()
    ax.barh([FEATURE_COLS[i] for i in sorted_idx], pi.importances_mean[sorted_idx])
    ax.set_title(name, fontsize=11)
    ax.set_xlabel('Mean decrease in score')
plt.tight_layout()
plt.show()

## 8. Prepare Test Set & Predict

In [ ]:
landsat_test = pd.read_csv("landsat_features_validation.csv")
terra_test   = pd.read_csv("terraclimate_features_validation.csv")
submission_template = pd.read_csv("submission_template.csv")

test = landsat_test.merge(terra_test, on=JOIN_KEYS, how='left')

# Same calibration as training
test = calibrate_landsat(test)

# Cloud filter
for col in BANDS:
    test[col] = test[col].where(test[col].between(-0.05, 1.2), np.nan)

# Impute using TRAINING cluster medians (assign test points to nearest train cluster)
test['geo_cluster'] = kmeans.predict(test[['Latitude', 'Longitude']])
for col in impute_cols:
    cluster_med = df.groupby('geo_cluster')[col].median()
    mask = test[col].isna()
    test.loc[mask, col] = test.loc[mask, 'geo_cluster'].map(cluster_med)
test[impute_cols] = test[impute_cols].fillna(train_medians)

# Feature engineering (same as training — no year, no PET lags)
dates_t = pd.to_datetime(test['Sample Date'], dayfirst=True)
test['month_sin'] = np.sin(2 * np.pi * dates_t.dt.month / 12)
test['month_cos'] = np.cos(2 * np.pi * dates_t.dt.month / 12)
test['NIR_SWIR22_ratio'] = test['nir'] / (test['swir22'] + 1e-6)
test['turbidity']        = test['swir16'] / (test['green'] + 1e-6)

X_test = test[FEATURE_COLS]
print(f"Test shape: {X_test.shape}")

# Predict (log-space → original scale)
pred_TA  = np.expm1(model_TA.predict(X_test))
pred_EC  = np.expm1(model_EC.predict(X_test))
pred_DRP = np.expm1(model_DRP.predict(X_test))

# Align predictions back to submission_template order
test['pred_TA']  = pred_TA
test['pred_EC']  = pred_EC
test['pred_DRP'] = pred_DRP

submission_df = submission_template[JOIN_KEYS].merge(
    test[JOIN_KEYS + ['pred_TA', 'pred_EC', 'pred_DRP']], on=JOIN_KEYS, how='left'
).rename(columns={
    'pred_TA': 'Total Alkalinity',
    'pred_EC': 'Electrical Conductance',
    'pred_DRP': 'Dissolved Reactive Phosphorus',
})

display(submission_df.head())
print(f"\nNaNs in submission: {submission_df.isna().sum().sum()}")

## 9. Save & Upload Submission

In [ ]:
submission_df.to_csv("/tmp/submission.csv", index=False)
print("Saved to /tmp/submission.csv")

In [ ]:
import snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

session.sql("""
    PUT file:///tmp/submission.csv
    'snow://workspace/USER$.PUBLIC."ey-hackathon"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()
print("File saved! Refresh the browser to see the files in the sidebar")